# Dataset Generation

## 1 - Connect to Vantage

In [1]:
import teradataml as tdml
import json
tdml.__version__

'20.00.00.03'

In [2]:
with open('../connections/me.json','r') as f:
    Param_ = json.load(f)

with open('../connections/csae2.json','r') as f:
    Param = json.load(f)

with open('../connections/csae.json','r') as f:
    Param_ = json.load(f)

with open('../connections/vantage24.json','r') as f:
    Param_ = json.load(f)

tdml.create_context(**Param)

Engine(teradatasql://:***@sto-xo0saiv0ot1sxmj2.env.clearscape.teradata.com?DATABASE=demo_user&user=DEMO_USER)

In [3]:
nb_amps = 1 + tdml.execute_sql('SEL hashamp()').fetchall()[0][0]
nb_amps

4

## 2 - Class definition

In [4]:
import tdstone2
tdstone2.__version__

'0.1.5.1'

In [5]:
from tdstone2.dataset_generation import GenerateDataSet,GenerateEquallyDistributedDataSet

In [ ]:
query, features = GenerateEquallyDistributedDataSet(n_partitions=nb_amps,n_rows=10000)

In [7]:
%%time
tdml.DataFrame.from_query(query).to_sql(schema_name=Param['database'],table_name='dataset_00',if_exists='replace',primary_index='Partition_ID')

CPU times: total: 15.6 ms
Wall time: 24 s


In [8]:
dataset = tdml.DataFrame(tdml.in_schema(Param['database'],'dataset_00'))
dataset

Partition_ID,ID,X1,X2,X3,X4,X5,X6,X7,X8,X9,flag,Y1,Y2,FOLD
6,351585,-0.1631493685821003,-1.096813154332773,0.27855174743059175,1.6467752963190412,-0.8772349384672065,0.016851962486606344,1.2638354664690692,1.2299006782940967,-0.20540627818450544,0,2.569565684620476,1,test
6,351588,0.5251759801143872,0.8326095200507567,0.986531250757179,1.0333165466772039,-0.9269581358776107,-0.7459250791183326,0.23331785217615544,0.7596969332265324,-1.1792289015863522,1,-0.7329629848050134,0,test
6,351589,1.741663004985227,-0.10949179330670253,-0.9395742818356816,0.13513927321301253,-0.9853538222089299,0.7835871923636013,-1.3699670034046576,0.7448244108263242,-0.1146818312167657,0,0.9441163908985271,1,test
6,351590,-1.5562357956459623,0.8959438053049256,-1.9475911202698495,0.06033497234510885,0.8539746496902315,-1.0775765421287427,0.7300455006283433,3.1299909327372433,1.9266837934461283,0,1.7005488297785,1,test
6,351592,-0.8780055058060817,-0.5031036664395013,-0.5180040055137646,-0.23179966683911502,-0.5830354184535343,-0.24272049968830822,-0.08095321138935771,0.7222836964060029,0.4527833684471396,0,-0.7917478455201481,0,test
6,351593,1.9504611316893363,-0.049105634471537284,-0.8937431216871966,0.565582924404196,-0.1549956781788508,0.6664174780027479,0.18194148668267793,-0.6927075693218121,1.0051127949497742,1,-2.4135120224468314,0,test
6,351591,0.6377353380106963,0.06014687448416787,-0.29237279071532246,-0.2720571633837945,-0.9840333479858863,0.5264558535424919,2.2599139821988485,-0.9577941585357472,-0.5858972645320824,0,0.4312020857269369,1,test
6,351586,0.3448311200897381,0.09027725144044174,0.9789947755282256,-0.7554943502794226,-2.266347485354949,-0.10268174465693217,-1.0566961882642933,1.0742263733757231,-0.6521917260501873,1,-1.698208674058414,0,test
6,351584,0.14230461848104087,-1.3197697455842938,1.3037056979431247,-0.5442128053599679,0.7384515683935469,-0.33681375759604143,-0.6986217022666343,0.5207189318127039,-1.2204139588221339,1,-4.217816642456988,0,test
6,351582,2.0974732869629453,-0.6413745371539084,0.7389199971410254,-0.7603125533892261,0.5730934353663517,-0.9064288123833675,0.2543157144004138,1.050968602372331,-0.21473054789463505,1,-1.0589261905149907,0,test


In [9]:
dataset.shape

(400000, 15)

In [10]:
summary = dataset[['Partition_ID','FOLD','ID']].groupby(['Partition_ID','FOLD']).count()
summary.sort(['Partition_ID','FOLD'])

Partition_ID,FOLD,count_ID
1,test,80000
1,train,20000
4,test,80000
4,train,20000
6,test,80000
6,train,20000
9,test,80000
9,train,20000


In [11]:
tdml.remove_context()

True